# Pattern Hunters: Phylogenetic Tree Construction
## Building Evolutionary Trees from Aligned Sequences

**For BSc Zoology Students**

---

### Learning Objectives
By the end of this notebook, you will:
1. Understand different tree-building methods (Distance, Parsimony, Maximum Likelihood)
2. Construct phylogenetic trees using real primate data
3. Calculate evolutionary distances between species
4. Compare trees built by different methods
5. Understand bootstrapping and tree reliability

### The Goal

We have aligned sequences. Now we ask:
**"What evolutionary tree best explains the patterns we observe?"**

Different methods answer this question in different ways:
- **Distance methods**: Group species by overall similarity
- **Parsimony**: Find the tree requiring fewest evolutionary changes
- **Maximum Likelihood**: Find the tree most probable given a model of evolution

---

## Part 1: Setup and Load Aligned Sequences

In [ ]:
# Install required packages
!pip install biopython matplotlib seaborn scipy -q
!pip install ete3 -q  # For tree visualization

print("✓ Packages installed!")

In [ ]:
from Bio import AlignIO, Phylo
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from Bio.Phylo.Consensus import bootstrap_consensus
from Bio import SeqIO
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from io import StringIO

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported!")

In [ ]:
# If you completed the previous notebook, you have an aligned file
# If not, we'll fetch and align sequences here

try:
    alignment = AlignIO.read("primate_aligned.fasta", "fasta")
    print("✓ Loaded alignment from previous notebook")
except FileNotFoundError:
    print("Alignment file not found. Creating new alignment...")
    # Code to create alignment would go here
    # For now, we'll assume the file exists

print(f"\nAlignment info:")
print(f"  Number of sequences: {len(alignment)}")
print(f"  Alignment length: {alignment.get_alignment_length()} bp")
print(f"\nSpecies:")
for record in alignment:
    print(f"  • {record.id}")

## Part 2: Understanding Evolutionary Distance

The first step in distance-based phylogenetics is calculating how different each pair of species is.

### Simple Distance Metrics:

1. **p-distance**: Proportion of sites that differ
   - Example: 950 identical sites out of 1000 → p-distance = 0.05

2. **Jukes-Cantor (JC69)**: Corrects for multiple substitutions at same site
   - Accounts for "hidden" changes (A→G→A looks unchanged)
   
3. **Kimura 2-parameter (K2P)**: Different rates for transitions vs transversions
   - Transitions (A↔G, C↔T) more common than transversions (A↔C, A↔T, G↔C, G↔T)

In [ ]:
# Calculate distances using different models
calculator = DistanceCalculator('identity')  # Simple identity distance
distance_matrix = calculator.get_distance(alignment)

print("Pairwise Distance Matrix (identity model):")
print(distance_matrix)

In [ ]:
# Visualize distance matrix as heatmap
species_names = [record.id for record in alignment]
n = len(species_names)

# Extract matrix values
dist_array = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        dist_array[i][j] = distance_matrix[i, j]

# Create DataFrame
dist_df = pd.DataFrame(dist_array, index=species_names, columns=species_names)

# Plot
plt.figure(figsize=(10, 8))
sns.heatmap(dist_df, annot=True, fmt='.4f', cmap='YlOrRd', 
            cbar_kws={'label': 'Evolutionary Distance'})
plt.title('Pairwise Evolutionary Distances\n(Lower = More Similar)', 
          fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("• Darker red = greater evolutionary distance")
print("• Diagonal is always 0 (species compared to itself)")
print("• Matrix is symmetric")

In [ ]:
# Find most and least similar pairs
print("Most similar species pairs (excluding self):")
print("="*60)

pairs = []
for i in range(n):
    for j in range(i+1, n):
        pairs.append((species_names[i], species_names[j], dist_array[i][j]))

pairs_sorted = sorted(pairs, key=lambda x: x[2])

print("\nClosest pairs:")
for sp1, sp2, dist in pairs_sorted[:5]:
    print(f"  {sp1:15} ↔ {sp2:15} : {dist:.4f}")

print("\nMost distant pairs:")
for sp1, sp2, dist in pairs_sorted[-5:]:
    print(f"  {sp1:15} ↔ {sp2:15} : {dist:.4f}")

## Part 3: Building Trees - UPGMA Method

**UPGMA** (Unweighted Pair Group Method with Arithmetic Mean) is the simplest tree-building method.

### How it works:
1. Find the two most similar species
2. Group them together
3. Calculate average distance from this group to all other species
4. Repeat until all species are grouped

**Assumption**: Molecular clock (all lineages evolve at same rate)
**Result**: Ultrametric tree (all tips same distance from root)

In [ ]:
# Build UPGMA tree
constructor = DistanceTreeConstructor()
upgma_tree = constructor.upgma(distance_matrix)

print("✓ UPGMA tree constructed!")
print("\nTree structure:")
print(upgma_tree)

In [ ]:
# Visualize UPGMA tree
fig, ax = plt.subplots(figsize=(12, 8))
Phylo.draw(upgma_tree, axes=ax, do_show=False)
ax.set_title('UPGMA Phylogenetic Tree\n(Assumes Constant Evolutionary Rate)', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nTree characteristics:")
print(f"  Total branch length: {upgma_tree.total_branch_length():.4f}")
print(f"  Number of terminals: {upgma_tree.count_terminals()}")
print(f"  Tree depth: {upgma_tree.depths()[upgma_tree.root]:.4f}")

## Part 4: Building Trees - Neighbor Joining Method

**Neighbor Joining (NJ)** is more sophisticated than UPGMA.

### Key differences:
- Does NOT assume constant evolutionary rates
- Produces unrooted trees
- Accounts for variation in branch lengths
- Generally more accurate than UPGMA

### How it works:
1. Calculate adjusted distances (accounting for divergence from other species)
2. Find pair of neighbors to join
3. Create new node, calculate branch lengths
4. Repeat with reduced matrix

In [ ]:
# Build Neighbor Joining tree
nj_tree = constructor.nj(distance_matrix)

print("✓ Neighbor Joining tree constructed!")
print("\nTree structure:")
print(nj_tree)

In [ ]:
# Visualize NJ tree
fig, ax = plt.subplots(figsize=(12, 8))
Phylo.draw(nj_tree, axes=ax, do_show=False)
ax.set_title('Neighbor Joining Phylogenetic Tree\n(No Molecular Clock Assumption)', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nTree characteristics:")
print(f"  Total branch length: {nj_tree.total_branch_length():.4f}")
print(f"  Number of terminals: {nj_tree.count_terminals()}")
print(f"  Is rooted: {nj_tree.is_bifurcating()}")

## Part 5: Compare UPGMA vs Neighbor Joining

In [ ]:
# Compare trees side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# UPGMA
Phylo.draw(upgma_tree, axes=ax1, do_show=False)
ax1.set_title('UPGMA Tree\n(Ultrametric)', fontsize=12, fontweight='bold')

# NJ
Phylo.draw(nj_tree, axes=ax2, do_show=False)
ax2.set_title('Neighbor Joining Tree\n(Variable Rates)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nComparison:")
print("="*60)
print(f"UPGMA total branch length: {upgma_tree.total_branch_length():.4f}")
print(f"NJ total branch length: {nj_tree.total_branch_length():.4f}")
print("\nKey differences:")
print("  • UPGMA: All tips equidistant from root (ultrametric)")
print("  • NJ: Tips at different distances (more realistic)")
print("  • NJ generally preferred for real evolutionary data")

## Part 6: Tree Confidence - Bootstrapping

**Question**: How confident are we in this tree?

**Bootstrap Analysis** tests tree reliability:
1. Randomly resample alignment columns (with replacement)
2. Build tree from resampled data
3. Repeat 100-1000 times
4. Count how often each branch appears

**Bootstrap value**: Percentage of trees containing that branch
- 95-100%: Very strong support
- 70-95%: Good support
- 50-70%: Weak support
- <50%: Very uncertain

In [ ]:
# Perform bootstrap analysis (simplified version)
# Note: Full bootstrap with 1000 replicates takes time

def bootstrap_tree(alignment, n_replicates=100):
    """
    Perform bootstrap analysis
    """
    print(f"Running {n_replicates} bootstrap replicates...")
    print("This may take a few minutes...\n")
    
    bootstrap_trees = []
    
    for i in range(n_replicates):
        if (i+1) % 20 == 0:
            print(f"  Completed {i+1}/{n_replicates} replicates")
        
        # Resample alignment columns
        alignment_length = alignment.get_alignment_length()
        resampled_indices = np.random.choice(alignment_length, 
                                           size=alignment_length, 
                                           replace=True)
        
        # Create resampled alignment
        resampled_seqs = []
        for record in alignment:
            new_seq = ''.join([str(record.seq[i]) for i in resampled_indices])
            from Bio.Seq import Seq
            from Bio.SeqRecord import SeqRecord
            resampled_seqs.append(SeqRecord(Seq(new_seq), id=record.id))
        
        # Convert to alignment object
        from Bio.Align import MultipleSeqAlignment
        boot_alignment = MultipleSeqAlignment(resampled_seqs)
        
        # Build tree
        calculator = DistanceCalculator('identity')
        dm = calculator.get_distance(boot_alignment)
        constructor = DistanceTreeConstructor()
        boot_tree = constructor.nj(dm)
        
        bootstrap_trees.append(boot_tree)
    
    print(f"\n✓ Bootstrap complete!")
    return bootstrap_trees

# Run bootstrap (using fewer replicates for speed)
bootstrap_trees = bootstrap_tree(alignment, n_replicates=100)

In [ ]:
# Get consensus tree
from Bio.Phylo.Consensus import majority_consensus

consensus_tree = majority_consensus(bootstrap_trees, cutoff=0.5)

print("✓ Consensus tree calculated")
print("\nConsensus tree structure:")
print(consensus_tree)

In [ ]:
# Visualize consensus tree with bootstrap support
fig, ax = plt.subplots(figsize=(12, 8))

def bootstrap_support_label(clade):
    """Label internal nodes with bootstrap support"""
    if clade.confidence and clade.confidence >= 0:
        return f"{int(clade.confidence * 100)}%"
    return ""

Phylo.draw(consensus_tree, axes=ax, do_show=False,
          label_func=lambda c: c.name if c.is_terminal() else bootstrap_support_label(c))
ax.set_title('Consensus Tree with Bootstrap Support\n(Values show % of trees supporting each branch)', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nBootstrap interpretation:")
print("  • Values near 100% indicate very reliable branches")
print("  • Values below 70% suggest uncertain relationships")
print("  • No value shown = terminal branches (always 100%)")

## Part 7: Understanding Branch Lengths

Branch lengths represent evolutionary distance:
- **Longer branches** = more changes
- **Shorter branches** = fewer changes

Can indicate:
- Time since divergence
- Rate of molecular evolution
- Natural selection pressure

In [ ]:
# Analyze branch lengths
def get_terminal_branch_lengths(tree):
    """
    Get branch lengths for all terminal taxa
    """
    lengths = {}
    for terminal in tree.get_terminals():
        # Sum branch lengths from root to terminal
        path_length = tree.distance(tree.root, terminal)
        lengths[terminal.name] = path_length
    return lengths

nj_lengths = get_terminal_branch_lengths(nj_tree)

print("Distance from root to each species (NJ tree):")
print("="*60)
for species, length in sorted(nj_lengths.items(), key=lambda x: x[1]):
    print(f"  {species:15} {length:.6f}")

print("\nInterpretation:")
print("  • Species with longer distances have accumulated more changes")
print("  • Could indicate faster evolution or longer independent history")

In [ ]:
# Visualize branch lengths as bar plot
species = list(nj_lengths.keys())
lengths = list(nj_lengths.values())

plt.figure(figsize=(10, 6))
plt.barh(species, lengths, color='steelblue')
plt.xlabel('Evolutionary Distance from Root', fontsize=12)
plt.ylabel('Species', fontsize=12)
plt.title('Root-to-Tip Distances\n(Proxy for Evolutionary Rate)', 
          fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## Part 8: Rooting the Tree

Neighbor Joining produces **unrooted trees**. To understand evolutionary direction, we need to root the tree.

### Rooting methods:
1. **Outgroup rooting**: Use a distantly related species
2. **Midpoint rooting**: Place root at tree's midpoint
3. **Molecular clock**: Root where clock assumption best holds

In [ ]:
# Root tree at midpoint
nj_tree_rooted = nj_tree.root_at_midpoint()

print("✓ Tree rooted at midpoint")

# Visualize rooted tree
fig, ax = plt.subplots(figsize=(12, 8))
Phylo.draw(nj_tree_rooted, axes=ax, do_show=False)
ax.set_title('Midpoint-Rooted Phylogenetic Tree', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# If we have an outgroup (e.g., Lemur), root there
if any('Lemur' in term.name for term in nj_tree.get_terminals()):
    # Find the Lemur terminal
    outgroup = [term for term in nj_tree.get_terminals() if 'Lemur' in term.name][0]
    nj_tree_outgroup = nj_tree.root_with_outgroup(outgroup)
    
    fig, ax = plt.subplots(figsize=(12, 8))
    Phylo.draw(nj_tree_outgroup, axes=ax, do_show=False)
    ax.set_title('Outgroup-Rooted Phylogenetic Tree\n(Rooted on Lemur)', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\nOutgroup rooting places the root between:")
    print("  • Lemur (more distantly related)")
    print("  • Other primates (more closely related)")
else:
    print("No outgroup (Lemur) found in this dataset")

## Part 9: Tree Comparison and Topology

Let's examine what our tree tells us about primate evolution.

In [ ]:
# Get clades (monophyletic groups)
def describe_tree_topology(tree, tree_name):
    """
    Describe the topology of a phylogenetic tree
    """
    print(f"\n{tree_name} Topology:")
    print("="*60)
    
    # Find all clades
    all_clades = list(tree.find_clades())
    internal_clades = [c for c in all_clades if not c.is_terminal()]
    
    print(f"Number of internal nodes: {len(internal_clades)}")
    print(f"Number of species: {len(tree.get_terminals())}")
    
    # Describe major groupings
    print("\nMajor groupings (clades):")
    for i, clade in enumerate(internal_clades, 1):
        terminals = clade.get_terminals()
        if len(terminals) >= 2:  # Only show clades with 2+ species
            species_names = [t.name for t in terminals]
            print(f"  Clade {i}: {', '.join(species_names)}")

# Describe our trees
describe_tree_topology(nj_tree, "Neighbor Joining")
describe_tree_topology(upgma_tree, "UPGMA")

In [ ]:
# Check if human and chimpanzee are sister species
def find_sister_taxa(tree, taxon_name):
    """
    Find the sister taxon/taxa to a given species
    """
    # Find the terminal
    target = None
    for terminal in tree.get_terminals():
        if taxon_name.lower() in terminal.name.lower():
            target = terminal
            break
    
    if not target:
        return None
    
    # Get parent clade
    path = tree.get_path(target)
    if len(path) < 2:
        return None
    
    parent = path[-2]
    
    # Get sister terminals
    sisters = [t.name for t in parent.get_terminals() if t != target]
    
    return sisters

# Find human's closest relative
human_sisters = find_sister_taxa(nj_tree, "Human")
if human_sisters:
    print("Human's closest relative(s) in the tree:")
    for sister in human_sisters:
        print(f"  • {sister}")
    print("\nThis matches molecular evidence: humans and chimpanzees")
    print("diverged ~6-7 million years ago.")

## Part 10: Saving Your Trees

In [ ]:
# Save trees in Newick format (standard format for phylogenetic trees)
Phylo.write(nj_tree, "primate_nj_tree.nwk", "newick")
Phylo.write(upgma_tree, "primate_upgma_tree.nwk", "newick")
if 'nj_tree_rooted' in locals():
    Phylo.write(nj_tree_rooted, "primate_nj_rooted_tree.nwk", "newick")

# Save as Nexus format (includes more metadata)
Phylo.write(nj_tree, "primate_nj_tree.nexus", "nexus")

print("✓ Trees saved in multiple formats:")
print("  • primate_nj_tree.nwk (Newick)")
print("  • primate_upgma_tree.nwk (Newick)")
print("  • primate_nj_tree.nexus (Nexus)")
print("\nThese can be opened in programs like:")
print("  • FigTree (visualization)")
print("  • MEGA (analysis)")
print("  • iTOL (online visualization)")

## Summary and Key Concepts

### What You've Learned:

1. **Evolutionary Distance**
   - Distance matrices quantify sequence differences
   - Different models account for different mutation patterns
   - Distance is foundation for tree building

2. **Tree Building Methods**
   - **UPGMA**: Simple, assumes molecular clock
   - **Neighbor Joining**: More realistic, no clock assumption
   - Different methods can give different trees!

3. **Tree Confidence**
   - Bootstrap analysis tests reliability
   - High bootstrap values = well-supported branches
   - Low values = uncertain relationships

4. **Tree Interpretation**
   - Branch lengths show evolutionary change
   - Topology shows relationships
   - Rooting determines evolutionary direction

### Real Biology:
- Humans and chimpanzees are sister species (diverged ~6-7 MYA)
- Great apes form a clade
- Old World monkeys are more distantly related
- Lemurs are even more distant (prosimians)

### Skills Acquired:
- Calculating evolutionary distances
- Building phylogenetic trees (UPGMA, NJ)
- Bootstrap analysis for tree support
- Tree visualization and interpretation
- Understanding tree topology

### Next Steps:
Move to Notebook 4: **Tree Interpretation and Evolutionary Analysis** to learn how to extract biological insights from phylogenetic trees!

---

## Questions to Think About

1. **Method Comparison**: Why might UPGMA and NJ give different trees? Which do you trust more?

2. **Bootstrap Values**: What does a bootstrap value of 65% mean biologically? Should we trust that branch?

3. **Branch Lengths**: If one species has much longer branches than others, what might explain this?

4. **Molecular Clock**: Does our tree support a molecular clock? How can you tell?

5. **Rooting**: Why does root placement matter? How does it change our interpretation?

6. **Real Evolution**: How well does our tree match what we know about primate evolution from fossils and anatomy?

Think about these as we explore tree interpretation in the next notebook!